In [1]:
import numpy as np
import pandas as pd
import re
import json
import requests as req
import warnings
warnings.simplefilter('ignore') 

df=pd.read_csv('games.csv')  # cargar csv en dataframe

df.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


CSV

In [2]:
df=df.drop(columns=['gameId', 'creationTime', 't1_ban1',      
                    't1_ban2', 't1_ban3', 't1_ban4' ,'t1_ban5',
                    't2_ban1', 't2_ban2', 't2_ban3', 't2_ban4', 't2_ban5'])

In [3]:
df.head()

,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,1949,9,1,2,1,1,1,1,2,8,...,14,4,412,4,3,5,0,0,1,1
1,1851,9,1,1,1,1,0,1,1,119,...,4,14,92,4,7,2,0,0,0,0
2,1493,9,1,2,1,1,1,2,0,18,...,4,11,22,7,4,2,0,0,1,0
3,1758,9,1,1,1,1,1,1,0,57,...,4,14,22,4,7,0,0,0,0,0
4,2094,9,1,2,1,1,1,1,0,19,...,4,12,51,4,7,3,0,0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gameDuration        51490 non-null  int64
 1   seasonId            51490 non-null  int64
 2   winner              51490 non-null  int64
 3   firstBlood          51490 non-null  int64
 4   firstTower          51490 non-null  int64
 5   firstInhibitor      51490 non-null  int64
 6   firstBaron          51490 non-null  int64
 7   firstDragon         51490 non-null  int64
 8   firstRiftHerald     51490 non-null  int64
 9   t1_champ1id         51490 non-null  int64
 10  t1_champ1_sum1      51490 non-null  int64
 11  t1_champ1_sum2      51490 non-null  int64
 12  t1_champ2id         51490 non-null  int64
 13  t1_champ2_sum1      51490 non-null  int64
 14  t1_champ2_sum2      51490 non-null  int64
 15  t1_champ3id         51490 non-null  int64
 16  t1_champ3_sum1      51490 non-null  int6

In [5]:
df=df.drop(columns=['t1_champ1_sum1', 't1_champ1_sum2', 't1_champ2_sum1', 't1_champ2_sum2','t1_champ3_sum1',
                    't1_champ3_sum2','t1_champ4_sum1' ,'t1_champ4_sum2' ,'t1_champ5_sum1', 't1_champ5_sum2',
                    't2_champ1_sum1','t2_champ1_sum2' ,'t2_champ2_sum1','t2_champ2_sum2','t2_champ3_sum1', 
                    't2_champ3_sum2','t2_champ4_sum1','t2_champ4_sum2','t2_champ5_sum1','t2_champ5_sum2'])

In [6]:
df.head()

,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,...,t2_champ1id,t2_champ2id,t2_champ3id,t2_champ4id,t2_champ5id,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,1949,9,1,2,1,1,1,1,2,8,...,104,498,122,238,412,5,0,0,1,1
1,1851,9,1,1,1,1,0,1,1,119,...,54,25,120,157,92,2,0,0,0,0
2,1493,9,1,2,1,1,1,2,0,18,...,69,412,126,24,22,2,0,0,1,0
3,1758,9,1,1,1,1,1,1,0,57,...,90,19,412,92,22,0,0,0,0,0
4,2094,9,1,2,1,1,1,1,0,19,...,37,59,141,38,51,3,0,0,1,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   gameDuration        51490 non-null  int64
 1   seasonId            51490 non-null  int64
 2   winner              51490 non-null  int64
 3   firstBlood          51490 non-null  int64
 4   firstTower          51490 non-null  int64
 5   firstInhibitor      51490 non-null  int64
 6   firstBaron          51490 non-null  int64
 7   firstDragon         51490 non-null  int64
 8   firstRiftHerald     51490 non-null  int64
 9   t1_champ1id         51490 non-null  int64
 10  t1_champ2id         51490 non-null  int64
 11  t1_champ3id         51490 non-null  int64
 12  t1_champ4id         51490 non-null  int64
 13  t1_champ5id         51490 non-null  int64
 14  t1_towerKills       51490 non-null  int64
 15  t1_inhibitorKills   51490 non-null  int64
 16  t1_baronKills       51490 non-null  int6

In [8]:
prueba1=df.copy()
type(prueba1)

pandas.core.frame.DataFrame

In [9]:
prueba1.rename(columns={'t1_champ1id': 'champid', 't1_champ2id': 'champid',
                   't1_champ3id': 'champid', 't1_champ4id': 'champid',
                  't1_champ5id': 'champid', 't2_champ1id': 'champid',
                  't2_champ2id': 'champid', 't2_champ3id': 'champid',
                  't2_champ4id': 'champid', 't2_champ5id': 'champid',}, inplace=True)



In [10]:
prueba1.head()

,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,champid,...,champid,champid,champid,champid,champid,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,1949,9,1,2,1,1,1,1,2,8,...,104,498,122,238,412,5,0,0,1,1
1,1851,9,1,1,1,1,0,1,1,119,...,54,25,120,157,92,2,0,0,0,0
2,1493,9,1,2,1,1,1,2,0,18,...,69,412,126,24,22,2,0,0,1,0
3,1758,9,1,1,1,1,1,1,0,57,...,90,19,412,92,22,0,0,0,0,0
4,2094,9,1,2,1,1,1,1,0,19,...,37,59,141,38,51,3,0,0,1,0


JSON

In [11]:
df2=pd.read_json('champion_info.json')
df2.head()

,type,version,data
1,champion,7.17.2,"{'title': 'the Dark Child', 'id': 1, 'key': 'A..."
10,champion,7.17.2,"{'title': 'The Judicator', 'id': 10, 'key': 'K..."
101,champion,7.17.2,"{'title': 'the Magus Ascendant', 'id': 101, 'k..."
102,champion,7.17.2,"{'title': 'the Half-Dragon', 'id': 102, 'key':..."
103,champion,7.17.2,"{'title': 'the Nine-Tailed Fox', 'id': 103, 'k..."


In [12]:
def flatten(data, cols):
    for c in cols:
        fl=pd.DataFrame(dict(data[c])).T
        cols=[str(col) for col in fl.columns]
        fl.columns=[c+'_'+name for name in cols]
        data=pd.concat([data, fl], axis=1)
        data=data.drop(c, axis=1)
    return data

In [13]:
nested_cols=['data']

flat=flatten(df2, nested_cols)

flat.head()

,type,version,data_title,data_id,data_key,data_name
1,champion,7.17.2,the Dark Child,1,Annie,Annie
10,champion,7.17.2,The Judicator,10,Kayle,Kayle
101,champion,7.17.2,the Magus Ascendant,101,Xerath,Xerath
102,champion,7.17.2,the Half-Dragon,102,Shyvana,Shyvana
103,champion,7.17.2,the Nine-Tailed Fox,103,Ahri,Ahri


In [14]:
flat.sort_values(by=['data_id'])
 

,type,version,data_title,data_id,data_key,data_name
1,champion,7.17.2,the Dark Child,1,Annie,Annie
2,champion,7.17.2,the Berserker,2,Olaf,Olaf
3,champion,7.17.2,the Colossus,3,Galio,Galio
4,champion,7.17.2,the Card Master,4,TwistedFate,Twisted Fate
5,champion,7.17.2,the Seneschal of Demacia,5,XinZhao,Xin Zhao
...,...,...,...,...,...,...
429,champion,7.17.2,the Spear of Vengeance,429,Kalista,Kalista
432,champion,7.17.2,the Wandering Caretaker,432,Bard,Bard
497,champion,7.17.2,The Charmer,497,Rakan,Rakan
498,champion,7.17.2,the Rebel,498,Xayah,Xayah


In [15]:
flat=flat.drop(columns=['type', 'version', 'data_title'])

In [16]:
flat=flat.drop(columns=['data_key'])

In [17]:
flat=flat.sort_values(by=['data_id'])
flat.head()

,data_id,data_name
1,1,Annie
2,2,Olaf
3,3,Galio
4,4,Twisted Fate
5,5,Xin Zhao


In [18]:
flat.rename(columns={'data_id': 'champid'}, inplace=True)

In [19]:
flat.head()

,champid,data_name
1,1,Annie
2,2,Olaf
3,3,Galio
4,4,Twisted Fate
5,5,Xin Zhao


In [20]:
flat.rename(columns={'data_name': 'Champ'}, inplace=True)

In [21]:
flat.head()

,champid,Champ
1,1,Annie
2,2,Olaf
3,3,Galio
4,4,Twisted Fate
5,5,Xin Zhao


WEB SCRAPPING

In [22]:
from bs4 import BeautifulSoup as bs

In [23]:
url='https://leagueoflegendsoficial.fandom.com/es/wiki/Datos_de_Campeones'
html=req.get(url).content
soup=bs(html, 'lxml')


In [24]:
data1 = pd.read_html('https://leagueoflegendsoficial.fandom.com/es/wiki/Datos_de_Campeones')[1]
data1

,Campeón,Edad,Altura,Peso
0,Aatrox,5000+,"3.65 m (12'0"")",317 kg (700 lb)
1,Ahri,35,"1.64 m (5'38"") (sin orejas) 5 cm (0'16"") (orejas)",60 kg (133 lb) 2 kg (4.4 lb) (cada cola)
2,Akali,22,"1.60 m (5'3"")",56 kg (125 lb)
3,Alistar,200,"3.17 m (10'5"")",680 kg (1500 lb)
4,Amumu,8000+,"1.09 m (3'7"")",15 kg (35 lb)
...,...,...,...,...
141,Ziggs,70,"76 cm (2'6"")",11 kg (30 lb)
142,Zilean,250,"1.85 m (6'1"")",74 kg (165 lb)
143,Zoe,1000+,"1.39 m (4'7"")",34 kg (75 lb)
144,Zyra,1700,"1.80 m (5'11"")",65 kg (145 lb)


In [25]:
data1.rename(columns={'Campeón': 'Champ'}, inplace=True)

In [26]:
data1.head()

,Champ,Edad,Altura,Peso
0,Aatrox,5000+,"3.65 m (12'0"")",317 kg (700 lb)
1,Ahri,35,"1.64 m (5'38"") (sin orejas) 5 cm (0'16"") (orejas)",60 kg (133 lb) 2 kg (4.4 lb) (cada cola)
2,Akali,22,"1.60 m (5'3"")",56 kg (125 lb)
3,Alistar,200,"3.17 m (10'5"")",680 kg (1500 lb)
4,Amumu,8000+,"1.09 m (3'7"")",15 kg (35 lb)


In [27]:
data2 = pd.read_html('https://leagueoflegendsoficial.fandom.com/es/wiki/Campeones')[1]
data2

,Campeón,Atributo,Lanzamiento,Rework,Esencia Azul,Riot Point
0,Aatrox,Luchador,13-06-2013,27-06-2018,6300,975
1,Ahri,Mago,14-12-2011,NaN,4800,880
2,Akali,Asesino,11-05-2010,31-07-2018 10-11-2016,3150,790
3,Alistar,Tanque,21-02-2009,14-01-2015 25-04-2011,1350,585
4,Amumu,Tanque,26-06-2009,NaN,450,260
...,...,...,...,...,...,...
140,Zed,Asesino,13-12-2011,10-11-2016,4800,880
141,Ziggs,Mago,01-02-2012,04-05-2016,4800,880
142,Zilean,Soporte,18-04-2009,25-02-2015,1350,585
143,Zoe,Mago,21-11-2017,NaN,6300,975


In [28]:
data2=data2.drop(['Rework'],axis=1)

In [29]:
data2.head()

,Campeón,Atributo,Lanzamiento,Esencia Azul,Riot Point
0,Aatrox,Luchador,13-06-2013,6300,975
1,Ahri,Mago,14-12-2011,4800,880
2,Akali,Asesino,11-05-2010,3150,790
3,Alistar,Tanque,21-02-2009,1350,585
4,Amumu,Tanque,26-06-2009,450,260


In [30]:
data2.rename(columns={'Campeón': 'Champ'}, inplace=True)

In [31]:
data2.tail()

,Champ,Atributo,Lanzamiento,Esencia Azul,Riot Point
140,Zed,Asesino,13-12-2011,4800,880
141,Ziggs,Mago,01-02-2012,4800,880
142,Zilean,Soporte,18-04-2009,1350,585
143,Zoe,Mago,21-11-2017,6300,975
144,Zyra,Mago,24-07-2012,4800,880


API

In [32]:
resp=req.get('http://ddragon.leagueoflegends.com/cdn/10.7.1/data/en_US/champion.json')
resp

<Response [200]>

In [33]:
results=resp.json()
results

{'data': {'Aatrox': {'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
   'id': 'Aatrox',
   'image': {'full': 'Aatrox.png',
    'group': 'champion',
    'h': 48,
    'sprite': 'champion0.png',
    'w': 48,
    'x': 0,
    'y': 0},
   'info': {'attack': 8, 'defense': 4, 'difficulty': 4, 'magic': 3},
   'key': '266',
   'name': 'Aatrox',
   'partype': 'Blood Well',
   'stats': {'armor': 38,
    'armorperlevel': 3.25,
    'attackdamage': 60,
    'attackdamageperlevel': 5,
    'attackrange': 175,
    'attackspeed': 0.651,
    'attackspeedperlevel': 2.5,
    'crit': 0,
    'critperlevel': 0,
    'hp': 580,
    'hpperlevel': 90,
    'hpregen': 3,
    'hpregenperlevel': 1,
    'movespeed': 345,
    'mp': 0,
    'mpperlevel': 0,
    'mpregen': 0,
    'mpregenperlevel': 0,
    's

In [34]:
api=pd.DataFrame(results)

api.head()

,type,format,version,data
Aatrox,champion,standAloneComplex,10.7.1,"{'version': '10.7.1', 'id': 'Aatrox', 'key': '..."
Ahri,champion,standAloneComplex,10.7.1,"{'version': '10.7.1', 'id': 'Ahri', 'key': '10..."
Akali,champion,standAloneComplex,10.7.1,"{'version': '10.7.1', 'id': 'Akali', 'key': '8..."
Alistar,champion,standAloneComplex,10.7.1,"{'version': '10.7.1', 'id': 'Alistar', 'key': ..."
Amumu,champion,standAloneComplex,10.7.1,"{'version': '10.7.1', 'id': 'Amumu', 'key': '3..."


In [35]:
nested_cols=['data','data_image']

lol1=flatten(api, nested_cols)

lol1.head()

,type,format,version,data_version,data_id,data_key,data_name,data_title,data_blurb,data_info,data_tags,data_partype,data_stats,data_image_full,data_image_sprite,data_image_group,data_image_x,data_image_y,data_image_w,data_image_h
Aatrox,champion,standAloneComplex,10.7.1,10.7.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","[Fighter, Tank]",Blood Well,"{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...",Aatrox.png,champion0.png,champion,0,0,48,48
Ahri,champion,standAloneComplex,10.7.1,10.7.1,Ahri,103,Ahri,the Nine-Tailed Fox,Innately connected to the latent power of Rune...,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","[Mage, Assassin]",Mana,"{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe...",Ahri.png,champion0.png,champion,48,0,48,48
Akali,champion,standAloneComplex,10.7.1,10.7.1,Akali,84,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...",[Assassin],Energy,"{'hp': 575, 'hpperlevel': 95, 'mp': 200, 'mppe...",Akali.png,champion0.png,champion,96,0,48,48
Alistar,champion,standAloneComplex,10.7.1,10.7.1,Alistar,12,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","[Tank, Support]",Mana,"{'hp': 600, 'hpperlevel': 106, 'mp': 350, 'mpp...",Alistar.png,champion0.png,champion,144,0,48,48
Amumu,champion,standAloneComplex,10.7.1,10.7.1,Amumu,32,Amumu,the Sad Mummy,Legend claims that Amumu is a lonely and melan...,"{'attack': 2, 'defense': 6, 'magic': 8, 'diffi...","[Tank, Mage]",Mana,"{'hp': 613.12, 'hpperlevel': 84, 'mp': 287.2, ...",Amumu.png,champion0.png,champion,192,0,48,48


In [36]:
lol1=lol1.drop(columns=['type', 'version', 'format','data_version','data_id','data_blurb','data_tags',
                        'data_partype','data_image_group','data_image_x','data_image_y',
                        'data_image_w','data_image_h'])

In [37]:
lol1.head()

,data_key,data_name,data_title,data_info,data_stats,data_image_full,data_image_sprite
Aatrox,266,Aatrox,the Darkin Blade,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...",Aatrox.png,champion0.png
Ahri,103,Ahri,the Nine-Tailed Fox,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe...",Ahri.png,champion0.png
Akali,84,Akali,the Rogue Assassin,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'hp': 575, 'hpperlevel': 95, 'mp': 200, 'mppe...",Akali.png,champion0.png
Alistar,12,Alistar,the Minotaur,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'hp': 600, 'hpperlevel': 106, 'mp': 350, 'mpp...",Alistar.png,champion0.png
Amumu,32,Amumu,the Sad Mummy,"{'attack': 2, 'defense': 6, 'magic': 8, 'diffi...","{'hp': 613.12, 'hpperlevel': 84, 'mp': 287.2, ...",Amumu.png,champion0.png


In [38]:
lol1=lol1.reset_index(drop=True)

In [39]:
lol1.head()

,data_key,data_name,data_title,data_info,data_stats,data_image_full,data_image_sprite
0,266,Aatrox,the Darkin Blade,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...",Aatrox.png,champion0.png
1,103,Ahri,the Nine-Tailed Fox,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe...",Ahri.png,champion0.png
2,84,Akali,the Rogue Assassin,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'hp': 575, 'hpperlevel': 95, 'mp': 200, 'mppe...",Akali.png,champion0.png
3,12,Alistar,the Minotaur,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'hp': 600, 'hpperlevel': 106, 'mp': 350, 'mpp...",Alistar.png,champion0.png
4,32,Amumu,the Sad Mummy,"{'attack': 2, 'defense': 6, 'magic': 8, 'diffi...","{'hp': 613.12, 'hpperlevel': 84, 'mp': 287.2, ...",Amumu.png,champion0.png


In [40]:
nested_cols=['data_info','data_stats']

lol2=flatten(lol1, nested_cols)

lol2.head()

,data_key,data_name,data_title,data_image_full,data_image_sprite,data_info_attack,data_info_defense,data_info_magic,data_info_difficulty,data_stats_hp,...,data_stats_hpregen,data_stats_hpregenperlevel,data_stats_mpregen,data_stats_mpregenperlevel,data_stats_crit,data_stats_critperlevel,data_stats_attackdamage,data_stats_attackdamageperlevel,data_stats_attackspeedperlevel,data_stats_attackspeed
0,266,Aatrox,the Darkin Blade,Aatrox.png,champion0.png,8,4,3,4,580.00,...,3.0,1.00,0.000,0.000,0.0,0.0,60.00,5.00,2.500,0.651
1,103,Ahri,the Nine-Tailed Fox,Ahri.png,champion0.png,3,4,8,5,526.00,...,6.5,0.60,8.000,0.800,0.0,0.0,53.04,3.00,2.000,0.668
2,84,Akali,the Rogue Assassin,Akali.png,champion0.png,5,3,8,7,575.00,...,8.0,0.50,50.000,0.000,0.0,0.0,62.40,3.30,3.200,0.625
3,12,Alistar,the Minotaur,Alistar.png,champion0.png,6,9,5,7,600.00,...,8.5,0.85,8.500,0.800,0.0,0.0,62.00,3.75,2.125,0.625
4,32,Amumu,the Sad Mummy,Amumu.png,champion0.png,2,6,8,3,613.12,...,9.0,0.85,7.382,0.525,0.0,0.0,53.38,3.80,2.180,0.736


In [41]:
lol2=lol2.drop(columns=['data_image_full','data_image_sprite','data_stats_critperlevel','data_stats_crit',])

In [42]:
lol2.rename(columns={'data_name': 'Champ'}, inplace=True)

In [43]:
lol2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   data_key                         148 non-null    object 
 1   Champ                            148 non-null    object 
 2   data_title                       148 non-null    object 
 3   data_info_attack                 148 non-null    int64  
 4   data_info_defense                148 non-null    int64  
 5   data_info_magic                  148 non-null    int64  
 6   data_info_difficulty             148 non-null    int64  
 7   data_stats_hp                    148 non-null    float64
 8   data_stats_hpperlevel            148 non-null    float64
 9   data_stats_mp                    148 non-null    float64
 10  data_stats_mpperlevel            148 non-null    float64
 11  data_stats_movespeed             148 non-null    float64
 12  data_stats_armor      

In [44]:
#lol2 df de estadisticas de champs

#data1 df de caracteritisticas fisicas de champs

#data2 df de precio y lanzamiento de champs

test_1=lol2.copy()
test_11=data1.copy()
test_111=data2.copy()

In [45]:
result = pd.merge(test_11, test_111, on='Champ')

In [46]:
result.head()

,Champ,Edad,Altura,Peso,Atributo,Lanzamiento,Esencia Azul,Riot Point
0,Aatrox,5000+,"3.65 m (12'0"")",317 kg (700 lb),Luchador,13-06-2013,6300,975
1,Ahri,35,"1.64 m (5'38"") (sin orejas) 5 cm (0'16"") (orejas)",60 kg (133 lb) 2 kg (4.4 lb) (cada cola),Mago,14-12-2011,4800,880
2,Akali,22,"1.60 m (5'3"")",56 kg (125 lb),Asesino,11-05-2010,3150,790
3,Alistar,200,"3.17 m (10'5"")",680 kg (1500 lb),Tanque,21-02-2009,1350,585
4,Amumu,8000+,"1.09 m (3'7"")",15 kg (35 lb),Tanque,26-06-2009,450,260


In [47]:
result2=pd.merge(result, test_1, on='Champ')

In [48]:
result2.head()

,Champ,Edad,Altura,Peso,Atributo,Lanzamiento,Esencia Azul,Riot Point,data_key,data_title,...,data_stats_spellblockperlevel,data_stats_attackrange,data_stats_hpregen,data_stats_hpregenperlevel,data_stats_mpregen,data_stats_mpregenperlevel,data_stats_attackdamage,data_stats_attackdamageperlevel,data_stats_attackspeedperlevel,data_stats_attackspeed
0,Aatrox,5000+,"3.65 m (12'0"")",317 kg (700 lb),Luchador,13-06-2013,6300,975,266,the Darkin Blade,...,1.25,175.0,3.0,1.00,0.000,0.000,60.00,5.00,2.500,0.651
1,Ahri,35,"1.64 m (5'38"") (sin orejas) 5 cm (0'16"") (orejas)",60 kg (133 lb) 2 kg (4.4 lb) (cada cola),Mago,14-12-2011,4800,880,103,the Nine-Tailed Fox,...,0.50,550.0,6.5,0.60,8.000,0.800,53.04,3.00,2.000,0.668
2,Akali,22,"1.60 m (5'3"")",56 kg (125 lb),Asesino,11-05-2010,3150,790,84,the Rogue Assassin,...,1.25,125.0,8.0,0.50,50.000,0.000,62.40,3.30,3.200,0.625
3,Alistar,200,"3.17 m (10'5"")",680 kg (1500 lb),Tanque,21-02-2009,1350,585,12,the Minotaur,...,1.25,125.0,8.5,0.85,8.500,0.800,62.00,3.75,2.125,0.625
4,Amumu,8000+,"1.09 m (3'7"")",15 kg (35 lb),Tanque,26-06-2009,450,260,32,the Sad Mummy,...,1.25,125.0,9.0,0.85,7.382,0.525,53.38,3.80,2.180,0.736


INSIGHTS

In [49]:
result2.sort_values(by=['Esencia Azul']).head()

,Champ,Edad,Altura,Peso,Atributo,Lanzamiento,Esencia Azul,Riot Point,data_key,data_title,...,data_stats_spellblockperlevel,data_stats_attackrange,data_stats_hpregen,data_stats_hpregenperlevel,data_stats_mpregen,data_stats_mpregenperlevel,data_stats_attackdamage,data_stats_attackdamageperlevel,data_stats_attackspeedperlevel,data_stats_attackspeed
101,Sivir,23,"1.72 m (5'8"")",63 kg (140 lb),Tirador,21-02-2009,450,260,15,the Battle Mistress,...,0.50,500.0,3.25,0.55,8.000,0.900,63.00,3.0,1.6,0.625
20,Dr. Mundo,18,"2.64 m (8'88"")",498 kg (1100 lb),Luchador,02-09-2009,450,260,36,the Madman of Zaun,...,1.25,125.0,8.00,0.75,0.000,0.000,61.27,3.5,2.8,0.721
83,Poppy,250,"96 cm (3'2"")",22 kg (50 lb),Tanque,13-01-2010,450,260,78,Keeper of the Hammer,...,1.25,125.0,8.00,0.80,7.000,0.700,64.00,4.0,2.5,0.625
129,Warwick,32,"2.33 m (7'8"")",201 kg (445 lb),Luchador,21-02-2009,450,260,19,the Uncaged Wrath of Zaun,...,1.25,125.0,4.00,0.75,7.466,0.575,65.00,3.0,2.3,0.638
52,Kayle,1000-2000,"1.65 m (5'5"")",59 kg (130 lb) 30 kg (66 lb) (Armadura),Luchador,21-02-2009,450,260,10,the Righteous,...,0.50,175.0,5.00,0.50,8.000,0.800,50.00,2.5,1.5,0.625


In [50]:
result2.sort_values(by=['Esencia Azul']).tail()

,Champ,Edad,Altura,Peso,Atributo,Lanzamiento,Esencia Azul,Riot Point,data_key,data_title,...,data_stats_spellblockperlevel,data_stats_attackrange,data_stats_hpregen,data_stats_hpregenperlevel,data_stats_mpregen,data_stats_mpregenperlevel,data_stats_attackdamage,data_stats_attackdamageperlevel,data_stats_attackspeedperlevel,data_stats_attackspeed
44,Jhin,31,"1.93 m (6'4"")",72 kg (160 lb),Tirador,01-02-2016,6300,975,202,the Virtuoso,...,0.50,550.0,3.75,0.55,6.00,0.80,59.0,4.7,0.0,0.625
81,Ornn,10000+,"9.14 m (30'0"")",4500 kg (9920 lb),Tanque,23-08-2017,6300,975,516,The Fire below the Mountain,...,1.25,175.0,9.00,0.90,8.01,0.60,69.0,3.5,2.0,0.625
39,Ivern,900,"3.04 m (10'0"")",181 kg (400 lb),Soporte,05-10-2016,6300,975,427,the Green Father,...,1.25,475.0,7.00,0.85,6.00,0.75,50.0,3.0,3.4,0.644
32,Gnar,1000+,"73 cm (2'5"") (Mini) 8.53 m (28'0"") (Mega)",11 kg (25 lb) (Mini) 2700 kg (5952 lb) (Mega),Luchador,14-08-2014,6300,975,150,the Missing Link,...,0.50,175.0,4.50,1.75,0.00,0.00,59.0,3.0,6.0,0.625
0,Aatrox,5000+,"3.65 m (12'0"")",317 kg (700 lb),Luchador,13-06-2013,6300,975,266,the Darkin Blade,...,1.25,175.0,3.00,1.00,0.00,0.00,60.0,5.0,2.5,0.651


In [51]:
result2.rename(columns={'data_key': 'Id','data_title':'Titulo'}, inplace=True)

In [52]:
result2.columns

Index(['Champ', 'Edad', 'Altura', 'Peso', 'Atributo', 'Lanzamiento',
       'Esencia Azul', 'Riot Point', 'Id', 'Titulo', 'data_info_attack',
       'data_info_defense', 'data_info_magic', 'data_info_difficulty',
       'data_stats_hp', 'data_stats_hpperlevel', 'data_stats_mp',
       'data_stats_mpperlevel', 'data_stats_movespeed', 'data_stats_armor',
       'data_stats_armorperlevel', 'data_stats_spellblock',
       'data_stats_spellblockperlevel', 'data_stats_attackrange',
       'data_stats_hpregen', 'data_stats_hpregenperlevel',
       'data_stats_mpregen', 'data_stats_mpregenperlevel',
       'data_stats_attackdamage', 'data_stats_attackdamageperlevel',
       'data_stats_attackspeedperlevel', 'data_stats_attackspeed'],
      dtype='object')

In [53]:
x=result2.copy()

In [54]:
x.columns=['Champ', 'Edad', 'Altura', 'Peso', 'Atributo', 'Lanzamiento',
       'Esencia Azul', 'Riot Point', 'Id', 'Titulo', 'attack',
       'defense', 'magic', 'difficulty',
       'hp', 'hpperlevel', 'mp',
       'mpperlevel', 'movespeed', 'armor',
       'armorperlevel', 'spellblock',
       'spellblockperlevel', 'attackrange',
       'hpregen', 'hpregenperlevel',
       'mpregen', 'mpregenperlevel',
       'attackdamage', 'attackdamageperlevel',
       'attackspeedperlevel', 'attackspeed']
    
        
        

In [55]:
final=x.copy()

In [56]:
final.armor.max()

47.0

In [57]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 142
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Champ                 143 non-null    object 
 1   Edad                  143 non-null    object 
 2   Altura                143 non-null    object 
 3   Peso                  143 non-null    object 
 4   Atributo              142 non-null    object 
 5   Lanzamiento           143 non-null    object 
 6   Esencia Azul          143 non-null    int64  
 7   Riot Point            143 non-null    int64  
 8   Id                    143 non-null    object 
 9   Titulo                143 non-null    object 
 10  attack                143 non-null    int64  
 11  defense               143 non-null    int64  
 12  magic                 143 non-null    int64  
 13  difficulty            143 non-null    int64  
 14  hp                    143 non-null    float64
 15  hpperlevel            1

In [58]:
final.describe()

,Esencia Azul,Riot Point,attack,defense,magic,difficulty,hp,hpperlevel,mp,mpperlevel,...,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,...,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,3924.125874,784.510490,5.433566,4.741259,5.454545,5.720280,555.559441,88.916084,309.898042,32.741259,...,0.895455,323.111888,6.634615,0.667832,8.445839,0.561014,59.531594,3.260710,2.473615,0.647762
std,1916.944946,205.454936,2.782135,2.095311,2.726312,2.050097,37.320555,7.098399,116.989719,16.675851,...,0.383319,195.758550,1.773923,0.162767,8.438498,0.284791,6.068835,0.589233,0.833143,0.035216
min,450.000000,260.000000,0.000000,1.000000,1.000000,1.000000,340.000000,65.000000,0.000000,0.000000,...,0.300000,125.000000,2.500000,0.400000,0.000000,0.000000,40.368000,1.500000,0.000000,0.475000
25%,3150.000000,790.000000,3.000000,3.000000,3.000000,4.000000,534.000000,85.000000,280.000000,25.000000,...,0.500000,125.000000,5.500000,0.550000,6.972000,0.425000,55.000000,3.000000,2.000000,0.625000
50%,4800.000000,880.000000,6.000000,5.000000,6.000000,6.000000,561.200000,90.000000,325.000000,38.000000,...,1.250000,175.000000,7.000000,0.600000,7.840000,0.650000,60.000000,3.200000,2.500000,0.644000
75%,4800.000000,880.000000,8.000000,6.000000,8.000000,7.000000,581.500000,92.500000,376.100000,44.500000,...,1.250000,525.000000,8.000000,0.750000,8.005000,0.800000,64.000000,3.500000,3.050000,0.661500
max,6300.000000,975.000000,10.000000,10.000000,10.000000,10.000000,625.640000,115.000000,500.000000,67.000000,...,1.750000,650.000000,10.000000,1.750000,50.000000,1.000000,70.000000,5.000000,6.000000,0.800000


In [59]:
final[(final.Atributo=='Tirador') & (final.difficulty<5)][['Champ', 'Titulo']]

,Champ,Titulo
7,Ashe,the Frost Archer
34,Graves,the Outlaw
56,Kindred,The Eternal Hunters
69,Miss Fortune,the Bounty Hunter
101,Sivir,the Battle Mistress
114,Tristana,the Yordle Gunner
121,Varus,the Arrow of Retribution


In [60]:
final[(final.Atributo=='Tanque') & (final.hp>600)][['Champ', 'Titulo']]

,Champ,Titulo
4,Amumu,the Sad Mummy
137,Zac,the Secret Weapon


In [61]:
final[(final.Atributo=='Asesino') & (final.attackdamage>64)][['Champ','Titulo']]

,Champ,Titulo
91,Rengar,the Pridestalker
110,Talon,the Blade's Shadow


In [62]:
final[(final.hp>600)][['Champ','Titulo']]

,Champ,Titulo
4,Amumu,the Sad Mummy
31,Garen,The Might of Demacia
51,Katarina,the Sinister Blade
102,Skarner,the Crystal Vanguard
115,Trundle,the Troll King
116,Tryndamere,the Barbarian King
137,Zac,the Secret Weapon


In [63]:
final[(final.Atributo=='Asesino')][['Champ','Titulo']]

,Champ,Titulo
2,Akali,the Rogue Assassin
22,Ekko,the Boy Who Shattered Time
24,Evelynn,Agony's Embrace
28,Fizz,the Tidal Trickster
50,Kassadin,the Void Walker
51,Katarina,the Sinister Blade
55,Kha'Zix,the Voidreaver
59,LeBlanc,the Deceiver
76,Nidalee,the Bestial Huntress
77,Nocturne,the Eternal Nightmare


In [64]:
final[(final.Atributo=='Mago')][['Champ','Titulo']]

,Champ,Titulo
1,Ahri,the Nine-Tailed Fox
5,Anivia,the Cryophoenix
6,Annie,the Dark Child
8,Aurelion Sol,The Star Forger
9,Azir,the Emperor of the Sands
11,Brand,the Burning Vengeance
15,Cassiopeia,the Serpent's Embrace
26,Fiddlesticks,the Ancient Fear
36,Heimerdinger,the Revered Inventor
48,Karma,the Enlightened One


In [65]:
final[(final.Atributo=='Tirador')][['Champ','Titulo']]

,Champ,Titulo
7,Ashe,the Frost Archer
13,Caitlyn,the Sheriff of Piltover
17,Corki,the Daring Bombardier
21,Draven,the Glorious Executioner
25,Ezreal,the Prodigal Explorer
34,Graves,the Outlaw
44,Jhin,the Virtuoso
45,Jinx,the Loose Cannon
46,Kai'Sa,Daughter of the Void
47,Kalista,the Spear of Vengeance


In [67]:
final.Edad=final.Edad.apply(lambda x:re.sub('\D+','',x))

In [81]:
final.Altura=final.Altura.apply(lambda x:re.sub('\((.*?)\)','',x))

In [83]:
final.Peso=final.Peso.apply(lambda x:re.sub('\((.*?)\)','',x))

In [84]:
final.head()

,Champ,Edad,Altura,Peso,Atributo,Lanzamiento,Esencia Azul,Riot Point,Id,Titulo,...,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,Aatrox,5000,3.65 m,317 kg,Luchador,13-06-2013,6300,975,266,the Darkin Blade,...,1.25,175.0,3.0,1.00,0.000,0.000,60.00,5.00,2.500,0.651
1,Ahri,35,1.64 m 5 cm,60 kg 2 kg,Mago,14-12-2011,4800,880,103,the Nine-Tailed Fox,...,0.50,550.0,6.5,0.60,8.000,0.800,53.04,3.00,2.000,0.668
2,Akali,22,1.60 m,56 kg,Asesino,11-05-2010,3150,790,84,the Rogue Assassin,...,1.25,125.0,8.0,0.50,50.000,0.000,62.40,3.30,3.200,0.625
3,Alistar,200,3.17 m,680 kg,Tanque,21-02-2009,1350,585,12,the Minotaur,...,1.25,125.0,8.5,0.85,8.500,0.800,62.00,3.75,2.125,0.625
4,Amumu,8000,1.09 m,15 kg,Tanque,26-06-2009,450,260,32,the Sad Mummy,...,1.25,125.0,9.0,0.85,7.382,0.525,53.38,3.80,2.180,0.736


In [87]:
x=final[['Champ','Id']]

In [88]:
x.head()

,Champ,Id
0,Aatrox,266
1,Ahri,103
2,Akali,84
3,Alistar,12
4,Amumu,32


In [89]:
x.rename(columns={'Id': 'champid'}, inplace=True)

In [91]:
df.rename(columns={'t1_champ1id': 'champid'}, inplace=True)